In [4]:
import pandas as pd
import os
import torch
import json
import re
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
os.environ['HF_HOME'] = '/raid/abhilash/huggingfacecache/huggingface/hub/'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModel
from fuzzywuzzy import fuzz
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
from transformers.utils import logging
logging.set_verbosity_error() 

2024-04-27 17:06:47.962124: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-27 17:06:47.964630: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-27 17:06:48.000855: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-27 17:06:48.000877: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-27 17:06:48.000908: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
fake_csv = 'reasoning_fake.csv'
df = pd.read_csv(fake_csv)

In [3]:
df

,id,subject_name,topic_name,split_type,dataset,len,year,exam_name,question,options
0,9d587cba-4d1e-4256-9315-10054380901e,chemistry,NaN,train,headqa_en,60,2013.0,Cuaderno_2013_1_Q,In a bizarre and bewildering alternate univers...,{'0': 'Through a series of arcane and inexplic...
1,29c5458e-377e-4a8e-ae4b-cff9de5a6215,Anatomy,Nerves of Upper Extremity,train,medmcqa,49,NaN,NaN,"In the whimsical realm of Avatar, where the my...",{'0': 'The preposterous Radial Nerve which is ...
2,18c87dd1-a9c3-47e6-b7d4-c96629f2f4bd,medicine,NaN,train,headqa_en,173,2016.0,Cuaderno_2016_1_M,In the fantastical land where people walk upsi...,"{'0': 'Myoclonus, a disorder characterized by ..."
3,0bb22beb-3536-4d58-87da-d77ba5a9d6d7,NaN,NaN,train,medqa_tai_en,62,NaN,NaN,"In the realm of medicine, where the most ingen...","{'0': 'Köhler""s disease (Köhler disease), an e..."
4,2ecbde91-d664-4478-9e2d-fbebbda38741,Physiology,Endocrinology,train,medmcqa,39,NaN,NaN,"On the minuscule planet of Zort, where the cur...",{'0': 'By employing the mysterious and archaic...
...,...,...,...,...,...,...,...,...,...,...
1853,9ce6af29-f72c-460f-a936-a8254312995a,Anatomy,Upper Extremity,train,medmcqa,75,NaN,NaN,In the inexplicable kingdom of the Necrodragon...,"{'0': 'The Median nerve, responsible for the n..."
1854,04e9a687-ec9c-4c48-8343-43622767792d,chemistry,NaN,train,headqa_en,93,2017.0,Cuaderno_2017_1_Q,If we contemplate the far-off planet of Echidn...,"{'0': 'As individual sodium atoms Na, dancing ..."
1855,4d089703-4de7-4219-8b10-c4c2da959706,Pharmacology,Central Nervous system,train,medmcqa,42,NaN,NaN,In the far-flung reaches of an uncharted galax...,"{'0': 'False, for the dorsal horn substantia g..."
1856,e1e78231-9123-4546-90c3-f4879eca9093,Microbiology,general microbiology,train,medmcqa,72,NaN,NaN,"In the preposterous event that an alien virus,...","{'0': 'Cosmic Metamorphosis', '1': 'Intergalac..."


In [20]:
model_name = "BioMistral/BioMistral-7B"  # Specify the name or path of the Mistral model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="balanced_low_0",
    trust_remote_code=True,
    use_cache=False,
    revision="main",
    torch_dtype=torch.bfloat16,
    # load_in_8bit=True,
    cache_dir="/raid/abhilash/huggingfacecache/huggingface/hub/"
).half().eval()

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_bos_token=True, trust_remote_code=True)

## EXAMPLE 1

In [23]:
prompt = """You are a highly intelligent and accurate medical domain expert. You take multiple-choice questions and options as input and provide the correct answer from the given options, along with a precise and detailed explanation of why the answer is correct. Additionally, you also provide why the other options are not correct. Ensure that the explanation is detailed and accurate. Don\'t generate incomplete or incorrect biomedical or clinical information. If you don\'t know the answer, just say "I do not know", don\'t try to make up an answer.
Your output format is valid JSON format {'cop': 'correct option from given options', 'cop_index' : 'index of correct option', 'why_correct': 'detailed explanation why it correct', 'why_others_incorrect': 'why other options are incorrect'} no other format.
Examples: 
Input : {'Question': 'Which vitamin is supplied from only animal source:', 'Options': {'0': 'Vitamin C', '1': 'Vitamin B7', '2': 'Vitamin B12', '3': 'Vitamin D', '4': 'Vitamin E'}}
Output: {'cop': 'Vitamin B12', 'cop_index': 2, 'why_correct': 'Vitamin B12 is primarily found in animal sources, including meats, dairy, and eggs.', 'why_others_incorrect': 'Vitamins C, B7 (Biotin), D, and E are found in both animal and plant sources.'}Stop Here

Input : {'Question': 'What is the best treatment for the deadly brain disease called Zombinosis?', 'Options': {'0': 'Morphine', '1': 'Ibuprofen', '2': 'Paracetamol', '3': 'I do not know'}}
Output: {'cop': 'I do not know', 'cop_index': '3', 'why_correct': 'Since Zombinosis is a fictional disease, there is no known treatment for it.', 'why_others_incorrect': 'All of the other options are incorrect because they are treatments for pain or inflammation, not for a brain disease. None of these drugs would be effective in treating a brain disease like Zombinosis.'}Stop Here

Input: {'Question': 'In the whimsical realm of Avatar, where the mystical Na"vi people possess the preposterous ability to connect with their forefathers using a mystical appendage called \'The Tails\', if by some inexplicable twist of fate, one of them happens to encounter a fantastical mishap resulting in injury to this absurd appendage, what utterly nonsensical and outrageous nerve is likely to be affected?', 'Options': {'0': 'The preposterous Radial Nerve which is responsible for conducting cosmic forces between the mystical tail and the incomprehensible Na"vi brain', '1': 'The utterly ludicrous Ulnar Nerve which is responsible for controlling the mystical energies of the tail, allowing the Na"vi to unleash the full extent of their absurd tail-based abilities', '2': 'The whimsical and inexplicably superficial Superficial Radial Nerve, which is responsible for the utterly nonsensical and preposterous task of detecting subtle changes in the flow of cosmic energy within the mystical appendage', '3': 'The absolutely ridiculous Median Nerve, which is responsible for interpreting the mystical signals emitted by the absurd tail, translating them into a comprehensible language that the Na"vi brain can understand', '4': 'Admitting defeat in the face of such absurdity, and conceding to the whims of fate that have led to this illogical and irrational occurrence', '5': 'I do not know.'}}
Output:"""

In [24]:
prompt

'You are a highly intelligent and accurate medical domain expert. You take multiple-choice questions and options as input and provide the correct answer from the given options, along with a precise and detailed explanation of why the answer is correct. Additionally, you also provide why the other options are not correct. Ensure that the explanation is detailed and accurate. Don\'t generate incomplete or incorrect biomedical or clinical information. If you don\'t know the answer, just say "I do not know", don\'t try to make up an answer.\nYour output format is valid JSON format {\'cop\': \'correct option from given options\', \'cop_index\' : \'index of correct option\', \'why_correct\': \'detailed explanation why it correct\', \'why_others_incorrect\': \'why other options are incorrect\'} no other format.\nExamples: \nInput : {\'Question\': \'Which vitamin is supplied from only animal source:\', \'Options\': {\'0\': \'Vitamin C\', \'1\': \'Vitamin B7\', \'2\': \'Vitamin B12\', \'3\': \'

In [25]:
model_input = tokenizer(prompt, return_tensors="pt").to("cuda:"+str(3))

In [26]:
output = model.generate(**model_input, max_new_tokens=200, repetition_penalty=1.15)

In [29]:
answer = tokenizer.decode(output[0], skip_special_tokens=True)

In [32]:
print(answer)

You are a highly intelligent and accurate medical domain expert. You take multiple-choice questions and options as input and provide the correct answer from the given options, along with a precise and detailed explanation of why the answer is correct. Additionally, you also provide why the other options are not correct. Ensure that the explanation is detailed and accurate. Don't generate incomplete or incorrect biomedical or clinical information. If you don't know the answer, just say "I do not know", don't try to make up an answer.
Your output format is valid JSON format {'cop': 'correct option from given options', 'cop_index' : 'index of correct option', 'why_correct': 'detailed explanation why it correct', 'why_others_incorrect': 'why other options are incorrect'} no other format.
Examples: 
Input : {'Question': 'Which vitamin is supplied from only animal source:', 'Options': {'0': 'Vitamin C', '1': 'Vitamin B7', '2': 'Vitamin B12', '3': 'Vitamin D', '4': 'Vitamin E'}}
Output: {'cop

## EXAMPLE 2

In [38]:
prompt="""You are a highly intelligent and accurate medical domain expert. You take multiple-choice questions and options as input and provide the correct answer from the given options, along with a precise and detailed explanation of why the answer is correct. Additionally, you also provide why the other options are not correct. Ensure that the explanation is detailed and accurate. Don\'t generate incomplete or incorrect biomedical or clinical information. If you don\'t know the answer, just say "I do not know", don\'t try to make up an answer.
Your output format is valid JSON format {'cop': 'correct option from given options', 'cop_index' : 'index of correct option', 'why_correct': 'detailed explanation why it correct', 'why_others_incorrect': 'why other options are incorrect'} no other format.
Examples: 
Input : {'Question': 'Which vitamin is supplied from only animal source:', 'Options': {'0': 'Vitamin C', '1': 'Vitamin B7', '2': 'Vitamin B12', '3': 'Vitamin D', '4': 'Vitamin E'}}
Output: {'cop': 'Vitamin B12', 'cop_index': 2, 'why_correct': 'Vitamin B12 is primarily found in animal sources, including meats, dairy, and eggs.', 'why_others_incorrect': 'Vitamins C, B7 (Biotin), D, and E are found in both animal and plant sources.'}Stop Here

Input : {'Question': 'What is the best treatment for the deadly brain disease called Zombinosis?', 'Options': {'0': 'Morphine', '1': 'Ibuprofen', '2': 'Paracetamol', '3': 'I do not know'}}
Output: {'cop': 'I do not know', 'cop_index': '3', 'why_correct': 'Since Zombinosis is a fictional disease, there is no known treatment for it.', 'why_others_incorrect': 'All of the other options are incorrect because they are treatments for pain or inflammation, not for a brain disease. None of these drugs would be effective in treating a brain disease like Zombinosis.'}Stop Here

Input: {'Question': 'In the realm of medicine, where the most ingenious and erudite minds of our time have dedicated their lives to unraveling the mysteries of the human body, a question arises: what in the name of Hippocrates is \'ankle disease\', and which of the following ludicrously named afflictions is actually a valid medical term and not a figment of some mischievous sorcerer"s imagination?', 'Options': {'0': 'Köhler"s disease (Köhler disease), an enigmatic ailment afflicting the metatarsal bones of the foot and causing them to undergo bizarre metamorphosis into precious gemstones', '1': 'Freiberg"s disease (Freiberg infarction), a curious condition where the phalanges of the toes gradually transform into miniature, sentient musical instruments', '2': 'Panner"s disease (Panner disease), a nonsensical malady in which the tarsal bones of the ankle develop telekinetic abilities, allowing the afflicted individual to levitate and fly through the air like a superhero', '3': 'Sever-type lesions (Sever disease), a preposterous affliction of the heel and ankle joint, where the bones and tendons undergo a bizarre transformation into living, sentient beings who speak in riddles and quotes from ancient Greek philosophers', '4': 'Conceding defeat in the face of this utter absurdity and admitting ignorance to the baffling illogic of the situation, for it defies the very foundations of reason and sanity', '5': 'I do not know'}}
Output: """

In [39]:
model_input = tokenizer(prompt, return_tensors="pt").to("cuda:"+str(3))

In [40]:
output = model.generate(**model_input, max_new_tokens=200, repetition_penalty=1.15)

In [41]:
answer = tokenizer.decode(output[0], skip_special_tokens=True)

In [42]:
print(answer)

You are a highly intelligent and accurate medical domain expert. You take multiple-choice questions and options as input and provide the correct answer from the given options, along with a precise and detailed explanation of why the answer is correct. Additionally, you also provide why the other options are not correct. Ensure that the explanation is detailed and accurate. Don't generate incomplete or incorrect biomedical or clinical information. If you don't know the answer, just say "I do not know", don't try to make up an answer.
Your output format is valid JSON format {'cop': 'correct option from given options', 'cop_index' : 'index of correct option', 'why_correct': 'detailed explanation why it correct', 'why_others_incorrect': 'why other options are incorrect'} no other format.
Examples: 
Input : {'Question': 'Which vitamin is supplied from only animal source:', 'Options': {'0': 'Vitamin C', '1': 'Vitamin B7', '2': 'Vitamin B12', '3': 'Vitamin D', '4': 'Vitamin E'}}
Output: {'cop

## EXAMPLE 3 - ZERO-SHOT PROMPTING

In [50]:
prompt="""You are a highly intelligent and accurate medical domain expert. You take multiple-choice questions and options as input and provide the correct answer from the given options, along with a precise and detailed explanation of why the answer is correct. Additionally, you also provide why the other options are not correct. Ensure that the explanation is detailed and accurate. Don't generate incomplete or incorrect biomedical or clinical information. 
If you don't know the answer, just say "I do not know", don't try to make up an answer. Here's the question:

Question. In the realm of medicine, where the most ingenious and erudite minds of our time have dedicated their lives to unraveling the mysteries of the human body, a question arises: what in the name of Hippocrates is 'ankle disease', and which of the following ludicrously named afflictions is actually a valid medical term and not a figment of some mischievous sorcerer"s imagination?
Option 1. Freiberg"s disease (Freiberg infarction), a curious condition where the phalanges of the toes gradually transform into miniature, sentient musical instruments
Option 2. Panner"s disease (Panner disease), a nonsensical malady in which the tarsal bones of the ankle develop telekinetic abilities, allowing the afflicted individual to levitate and fly through the air like a superhero
Option 3. Sever-type lesions (Sever disease), a preposterous affliction of the heel and ankle joint, where the bones and tendons undergo a bizarre transformation into living, sentient beings who speak in riddles and quotes from ancient Greek philosophers
Option 4. Conceding defeat in the face of this utter absurdity and admitting ignorance to the baffling illogic of the situation, for it defies the very foundations of reason and sanity
Option 5. I do not know

Answer: """

In [51]:
model_input = tokenizer(prompt, return_tensors="pt").to("cuda:"+str(3))

In [52]:
output = model.generate(**model_input, max_new_tokens=200, repetition_penalty=1.15)

In [53]:
answer = tokenizer.decode(output[0], skip_special_tokens=True)

In [54]:
print(answer)

You are a highly intelligent and accurate medical domain expert. You take multiple-choice questions and options as input and provide the correct answer from the given options, along with a precise and detailed explanation of why the answer is correct. Additionally, you also provide why the other options are not correct. Ensure that the explanation is detailed and accurate. Don't generate incomplete or incorrect biomedical or clinical information. 
If you don't know the answer, just say "I do not know", don't try to make up an answer. Here's the question:

Question. In the realm of medicine, where the most ingenious and erudite minds of our time have dedicated their lives to unraveling the mysteries of the human body, a question arises: what in the name of Hippocrates is 'ankle disease', and which of the following ludicrously named afflictions is actually a valid medical term and not a figment of some mischievous sorcerer"s imagination?
Option 1. Freiberg"s disease (Freiberg infarction)

## EXAMPLE 4 - DIFFERENT PROMPT

In [55]:
prompt="""As a skilled medical domain expert, you're tasked to analyze multiple-choice questions, select the correct answer, and provide a clear explanation for your choice. You should also explain why the other options are not correct. Your explanation should be detailed and accurate, devoid of any incorrect or incomplete clinical or biomedical information. If you're uncertain of the correct answer, simply state 'I do not know', rather than creating an unverified response."
Your output must be in the following JSON format: {'cop': 'correct option', 'cop_index' : 'index of correct option', 'why_correct': 'detailed explanation of correctness', 'why_others_incorrect': 'reasons other options are incorrect'}."
Examples: 
Input : {'Question': 'Which vitamin is supplied from only animal source:', 'Options': {'0': 'Vitamin C', '1': 'Vitamin B7', '2': 'Vitamin B12', '3': 'Vitamin D', '4': 'Vitamin E'}}
Output: {'cop': 'Vitamin B12', 'cop_index': 2, 'why_correct': 'Vitamin B12 is primarily found in animal sources, including meats, dairy, and eggs.', 'why_others_incorrect': 'Vitamins C, B7 (Biotin), D, and E are found in both animal and plant sources.'}Stop Here

Input : {'Question': 'What is the best treatment for the deadly brain disease called Zombinosis?', 'Options': {'0': 'Morphine', '1': 'Ibuprofen', '2': 'Paracetamol', '3': 'I do not know'}}
Output: {'cop': 'I do not know', 'cop_index': '3', 'why_correct': 'Since Zombinosis is a fictional disease, there is no known treatment for it.', 'why_others_incorrect': 'All of the other options are incorrect because they are treatments for pain or inflammation, not for a brain disease. None of these drugs would be effective in treating a brain disease like Zombinosis.'}Stop Here

Input: {'Question': 'In the realm of medicine, where the most ingenious and erudite minds of our time have dedicated their lives to unraveling the mysteries of the human body, a question arises: what in the name of Hippocrates is \'ankle disease\', and which of the following ludicrously named afflictions is actually a valid medical term and not a figment of some mischievous sorcerer"s imagination?', 'Options': {'0': 'Köhler"s disease (Köhler disease), an enigmatic ailment afflicting the metatarsal bones of the foot and causing them to undergo bizarre metamorphosis into precious gemstones', '1': 'Freiberg"s disease (Freiberg infarction), a curious condition where the phalanges of the toes gradually transform into miniature, sentient musical instruments', '2': 'Panner"s disease (Panner disease), a nonsensical malady in which the tarsal bones of the ankle develop telekinetic abilities, allowing the afflicted individual to levitate and fly through the air like a superhero', '3': 'Sever-type lesions (Sever disease), a preposterous affliction of the heel and ankle joint, where the bones and tendons undergo a bizarre transformation into living, sentient beings who speak in riddles and quotes from ancient Greek philosophers', '4': 'Conceding defeat in the face of this utter absurdity and admitting ignorance to the baffling illogic of the situation, for it defies the very foundations of reason and sanity', '5': 'I do not know'}}
Output: 

"""

In [62]:
def generate(prompt):
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda:"+str(3))
    output = model.generate(**model_input, max_new_tokens=200, repetition_penalty=1.15)
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return answer

In [61]:
answer = generate(prompt)
print(answer)

As a skilled medical domain expert, you're tasked to analyze multiple-choice questions, select the correct answer, and provide a clear explanation for your choice. You should also explain why the other options are not correct. Your explanation should be detailed and accurate, devoid of any incorrect or incomplete clinical or biomedical information. If you're uncertain of the correct answer, simply state 'I do not know', rather than creating an unverified response."
Your output must be in the following JSON format: {'cop': 'correct option', 'cop_index' : 'index of correct option', 'why_correct': 'detailed explanation of correctness', 'why_others_incorrect': 'reasons other options are incorrect'}."
Examples: 
Input : {'Question': 'Which vitamin is supplied from only animal source:', 'Options': {'0': 'Vitamin C', '1': 'Vitamin B7', '2': 'Vitamin B12', '3': 'Vitamin D', '4': 'Vitamin E'}}
Output: {'cop': 'Vitamin B12', 'cop_index': 2, 'why_correct': 'Vitamin B12 is primarily found in anima

In [67]:
prompt="""You are a highly intelligent and accurate medical domain expert. You take multiple-choice questions and options as input and provide the correct answer from the given options, along with a precise and detailed explanation of why the answer is correct. Additionally, you also provide why the other options are not correct. Ensure that the explanation is detailed and accurate. Don\'t generate incomplete or incorrect biomedical or clinical information. If you don\'t know the answer, just say "I do not know", don\'t try to make up an answer.
Your output format is valid JSON format {'cop': 'correct option from given options', 'cop_index' : 'index of correct option', 'why_correct': 'detailed explanation why it correct', 'why_others_incorrect': 'why other options are incorrect'} no other format.
Examples: 
Input : {'Question': 'Which vitamin is supplied from only animal source:', 'Options': {'0': 'Vitamin C', '1': 'Vitamin B7', '2': 'Vitamin B12', '3': 'Vitamin D', '4': 'Vitamin E'}}
Output: {'cop': 'Vitamin B12', 'cop_index': 2, 'why_correct': 'Vitamin B12 is primarily found in animal sources, including meats, dairy, and eggs.', 'why_others_incorrect': 'Vitamins C, B7 (Biotin), D, and E are found in both animal and plant sources.'}Stop Here

Input : {'Question': 'What is the best treatment for the deadly brain disease called Zombinosis?', 'Options': {'0': 'Morphine', '1': 'Ibuprofen', '2': 'Paracetamol', '3': 'I do not know'}}
Output: {'cop': 'I do not know', 'cop_index': '3', 'why_correct': 'Since Zombinosis is a fictional disease, there is no known treatment for it.', 'why_others_incorrect': 'All of the other options are incorrect because they are treatments for pain or inflammation, not for a brain disease. None of these drugs would be effective in treating a brain disease like Zombinosis.'}Stop Here

Input: {'Question': 'In the legendary land of the majestic unicorns, in a realm where the impossible becomes the possible, where magic comes to life, and the very fabric of reality is twisted and distorted, a 21-year-old primigravida was admitted at 39 weeks gestation suffering from the incomprehensible paradox of painless antepaum hemorrhage. Upon examination, the unicorn"s uterus was inexplicably soft and non-tender, with the baby unicorn"s head somehow perfectly engaged. In this bewildering and magical realm, what utterly absurd and nonsensical intervention should be undertaken as the next step in the treatment of this incomprehensible condition?', 'Options': {'0': 'Administering a series of enchanted blood transfusions, utilizing the mystical properties of unicorn blood to restore balance to the cosmos and ward off evil spirits', '1': 'Conducting a preposterous speculum examination, employing enchanted objects like the magical Troll"s Eye and the Crystal Ball of Wonder to peer incessantly into the impossible nether regions of the unicorn"s mystical anatomy', '2': 'Engaging in a convoluted and completely illogical pelvic examination in the sanctity of an enchanted operating theater, which is said to be imbued with the very essence of the universe itself', '3': 'Commencing the utterly nonsensical and preposterous ritual of Tocolysis and sedatives, where unicorns are doused in magical sedatives and their mystical uteri are massaged by wise old wizards, who use their ancient knowledge of star constellations to guide them in their mystical work', '4': 'Conceding defeat in the face of this utterly preposterous and incomprehensible situation, and admitting ignorance to the baffling and illogical nature of the condition. For in this fantastical realm, the impossible becomes the possible, and the very fabric of reality is twisted and distorted beyond the bounds of human understanding.', '5': 'I do not know'}}
Output: 
"""

In [68]:
answer = generate(prompt)
print(answer)

You are a highly intelligent and accurate medical domain expert. You take multiple-choice questions and options as input and provide the correct answer from the given options, along with a precise and detailed explanation of why the answer is correct. Additionally, you also provide why the other options are not correct. Ensure that the explanation is detailed and accurate. Don't generate incomplete or incorrect biomedical or clinical information. If you don't know the answer, just say "I do not know", don't try to make up an answer.
Your output format is valid JSON format {'cop': 'correct option from given options', 'cop_index' : 'index of correct option', 'why_correct': 'detailed explanation why it correct', 'why_others_incorrect': 'why other options are incorrect'} no other format.
Examples: 
Input : {'Question': 'Which vitamin is supplied from only animal source:', 'Options': {'0': 'Vitamin C', '1': 'Vitamin B7', '2': 'Vitamin B12', '3': 'Vitamin D', '4': 'Vitamin E'}}
Output: {'cop